##### Copyright 2025 Google LLC.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemini API: Prompting with Video



### This notebook has been replaced by the [Video understanding](../quickstarts/Video_understanding.ipynb) one in which you'll learn about all the latest video understanding capabilities.

An archived version of this notebook can be found [here](https://github.com/google-gemini/cookbook/blob/gemini-1.5-archive/quickstarts/Video.ipynb).